In [ ]:
!pip install torchsummary

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms,datasets
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm
import os
import numpy as np
import cv2
import zipfile
reprocess_data = True
from PIL import Image
from torchsummary import summary
import time

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
! unzip ../input/dogs-vs-cats/test1.zip 
! unzip ../input/dogs-vs-cats/train.zip 

In [ ]:
train_path = "./train"
test_path ="./test1"
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

In [ ]:
print(len(train_files))
print(len(test_files))

In [ ]:
imgpath = os.path.join(train_path,train_files[0])
print(imgpath)

In [ ]:
img = cv2.imread(imgpath,0)
type(img)

In [ ]:
imgtensor = torch.tensor(img)

In [ ]:
imgtensor.shape

In [ ]:
# imgtensorresize = transforms.

In [ ]:
class Dataset():
    def __init__(self,filelist,filepath,transform = None):
        self.filelist = filelist
        self.filepath = filepath
        self.transform = transform
    def __len__(self):
        return int(len(self.filelist))
    def __getitem__(self,index):
        imgpath = os.path.join(self.filepath,self.filelist[index])
        img = Image.open(imgpath).convert('L')
        if "dog" in imgpath:
            label = 1
        else:
            label = 0 
        if self.transform is not None:
            img = self.transform(img)
        return (img,label)
            
        

In [ ]:
transformations = transforms.Compose([transforms.Resize((60,60)),transforms.ToTensor()])
train = Dataset(train_files,train_path,transformations)
test = Dataset(test_files,test_path,transformations)
train_set,val_set = torch.utils.data.random_split(train,[20000,5000]) 

In [ ]:
print("train set size :",train_set.__len__())
print("val set size :",val_set.__len__())

In [ ]:
train_set.__getitem__(0)[0].shape

In [ ]:
train_dataset = torch.utils.data.DataLoader(dataset = train,batch_size = 16,shuffle=True)
test_dataset = torch.utils.data.DataLoader(dataset = test,batch_size = 16,shuffle=True)

In [ ]:
class Covnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
            ) 
            
        self.conv2 =   nn.Sequential(
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        ) 
        self.conv3 =   nn.Sequential(
            nn.Conv2d(64,128,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
            )
        self.fc1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(2048,512),
        nn.ReLU()
        )
            
        self.fc2 = nn.Sequential(
        nn.Linear(512,2),
        )
                
    def forward(self,x):
        x= self.conv1(x)
        x= self.conv2(x)
        x= self.conv3(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return F.softmax(x,dim = 1) 
        


In [ ]:
model = Covnet()
model.cuda()
summary(model,(1,60,60))

In [ ]:
EPOCHS = 10
optimiser = optim.Adam(model.parameters(),lr=0.001)
LOSS = nn.CrossEntropyLoss()

In [ ]:
len(train_dataset)

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    total = 0
    correct = 0
    train_loss = 0
    train_accuracy = 0
    counter = 0
    with tqdm(train_dataset, unit="batch") as tepoch:
        tepoch.set_description(f'Epoch {epoch+1}/{EPOCHS}')
        for data,label in tepoch:
            data,label = data.to(device) , label.to(device)
            optimiser.zero_grad()
            output = model(data)
            loss = LOSS(output,label)
            loss.backward()
            optimiser.step() 
            train_loss+= loss.item() * data.size(0)
            _,pred = torch.max(output.data,1)
            total += label.size(0)
            correct += (pred == label).sum().item()
        train_accuracy = correct/total
#         tepoch.set_postfix(accuracy =train_accuracy,loss = train_loss/len(train_dataset))
        print("=====/train/=====")
        print("epoch {} train accuracy {}".format(epoch+1,train_accuracy))
        print("epoch {} train loss {}".format(epoch+1,train_loss/len(train_set)))
    if epoch %1 == 0:
        model.eval()
        total = 0
        correct = 0
        val_loss = 0
        val_accuarcy =0
        with torch.no_grad():
            for data,label in test_dataset:
                data,label = data.to(device),label.to(device)
                output = model(data)
                loss = LOSS(output,label)
                val_loss+= loss.item() * data.size(0)
                _,pred = torch.max(output.data,1)    
                total += label.size(0)
                correct += (pred == label).sum().item()
            val_accuarcy = correct/total
            print("=====/val/=====")
            print("epoch {} val accuracy {}".format(epoch+1,val_accuarcy))
            print("epoch {} val loss {}".format(epoch+1,val_loss/len(val_set)))
                    
                    

In [ ]:
print(0%1)